In [1]:
import itertools
import math
from sympy import *

In [41]:
def build_divisors(l):
    r, s = symbols('r s')
    divisors = [r**2+r+1, r**2*s**2 + r*s - s - 1, r**2+1, s**2-r, r*s**2-1, s+r+1, (r+1)*s+1, r*s+r+s, (r**2+r)*s+1, r**2+r*s+s, r-1, r+1, s**2+1, s**2+r, r*s**2+1, r**2+r+s, r*s+r+1, r**2*s+r+1, r**2 + r*s + 1, r**2*s + r + s  ]
    for i in range(l):
        divisors.append(s-r**i)
        divisors.append(s+r**i)
    for i in range(1,l):
        divisors.append(s*r**i+1)
        divisors.append(s*r**i-1)
    
    return divisors

In [5]:
def good_octuple_new(a,b,c,d,e,f,g,h,t,u,v,x,y,z):
    good = True
    if not (min(a,b,c,d)==min(e,f,g,h)==0):
        good = False
    if (u==0 and a==c) or (v==0 and a==d) or (t==u and b==c) or (t==v and b==d):
        good = False
    if (y==0 and e==g) or (z==0 and e==h) or (x==y and f==g) or (x==z and f==h):
        good = False
    if (t==0 and a<b) or (u==v and c<d) or (x==0 and e<f) or (y==z and g<h) or (u==0 and t==v and c>=a) or (y==0 and x==z and e<=g):
        good = False
        
    if (t==x==0 and {a,b}=={e,f}) or (u==v==y==z and {c,d}=={g,h}):
        good = False
    if ((0,t) == (u,v) == (0,x) == (y,z)) and (((a,b) == (e,f) and (c,d) == (g, h)) or ((a,b) == (g,h) and (c,d) == (e,f))):
        good = False

    if ((t,u,v)==(x,y,z) and a-e==b-f==c-g==d-h):
        good = False
    
    return good

In [63]:
def factor_out_new(divs,a,b,c,d,e,f,g,h,t,u,v,x,y,z,l):
    r, s = symbols('r s')
    p = r**(a)+s**t*r**(b)-s**u*r**(c)-s**v*r**(d)
    q = r**(e)+s**x*r**(f)-s**y*r**(g)-s**z*r**(h)
   
    G = gcd(p,q)   
    
    for div in divs:
        quot, rem = reduced(G, [div], [r, s])   
        if rem == 0:
            p = (p/div).cancel()
            q = (q/div).cancel()
           
    return p,q

In [9]:
ExponentTuples= [(0,1,1),(0,0,1),(1,1,1),(1,0,1),(0,2,2),(0,0,2),(2,2,2),(2,0,2),(1,1,2),(2,1,1),(1,2,2),(2,1,2),(0,1,2),(1,0,2)]

In [11]:
Bad_Dictionary = {}
for i in range(14):
    for j in range(i,14):
        (t,u,v) = ExponentTuples[i]
        (x,y,z) = ExponentTuples[j]
        Bad_Dictionary[(t,u,v,x,y,z)] = []

In [13]:
Collision_Dictionary = {}
for i in range(14):
    for j in range(i,14):
        (t,u,v) = ExponentTuples[i]
        (x,y,z) = ExponentTuples[j]
        Collision_Dictionary[(t,u,v,x,y,z)] = []

In [15]:
def get_rat_roots(p):
    rats = set([])
    if p.is_constant():
        return rats
    else: 
        roots = real_roots(p)
        for x in roots:
            if x.is_rational and x>1:
                rats.add(x)
        return rats
        

In [67]:
def alt_general_resultant(divs,t,u,v,x,y,z,l):
  bad = []
  collisions = []
  L = range(l)
  r, s = symbols('r s')
  for (a,b,c,d,e,f,g,h) in itertools.product(L,L,L,L,L,L,L,L):
          if good_octuple_new(a,b,c,d,e,f,g,h,t,u,v,x,y,z):
            p,q = factor_out_new(divs,a,b,c,d,e,f,g,h,t,u,v,x,y,z,l)
            if not p.is_constant() and not q.is_constant():
                res1 = resultant(p,q,s)
                res2 = resultant(p,q,r)
                if res1 == 0 or res2 == 0:
                    bad.append((a,b,c,d,e,f,g,h))
                else:
                    r_roots = get_rat_roots(res1)
                    s_roots = get_rat_roots(res2)
                    for r1 in r_roots:
                        for s1 in s_roots:
                            p1 = p.subs({r:r1,s:s1})
                            q1 = q.subs({r:r1,s:s1})
                            if r1 != s1 and r1**2 != s1 and r1 != s1**2 and p1 == q1 == 0:
                                collisions.append((r1,s1,a,b,c,d,e,f,g,h))
                
  return bad, collisions

In [73]:
def AltBigResultant(l):
    done = 0
    ETs = Collision_Dictionary.keys()
    divs = build_divisors(l)
    for (t,u,v,x,y,z) in ETs:
            bad,  collisions = alt_general_resultant(divs,t,u,v,x,y,z,l)
            if len(bad) != 0:
                print((t,u,v,x,y,z),' danger')
            Bad_Dictionary[(t,u,v,x,y,z)] = bad
            Collision_Dictionary[(t,u,v,x,y,z)] = collisions
            done = done+1
            print(done)
    

In [75]:
AltBigResultant(4)

1
(0, 1, 1, 0, 0, 1)  danger
2
(0, 1, 1, 1, 1, 1)  danger
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
(0, 2, 2, 0, 0, 2)  danger
52
(0, 2, 2, 2, 2, 2)  danger
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93



KeyboardInterrupt



In [77]:
Exception_List = set([])
for v in Collision_Dictionary.values():
    for (r,s,a,b,c,d,e,f,g,h) in v:
        Exception_List.add((r,s))

In [79]:
len(Exception_List)

72

In [87]:
Exception_List

{(5/4, 6/5),
 (5/4, 3/2),
 (5/4, 8/5),
 (5/4, 2),
 (5/4, 5/2),
 (4/3, 9/8),
 (4/3, 8/7),
 (4/3, 7/6),
 (4/3, 3/2),
 (4/3, 2),
 (4/3, 8/3),
 (3/2, 9/8),
 (3/2, 6/5),
 (3/2, 5/4),
 (3/2, 4/3),
 (3/2, 2),
 (3/2, 8/3),
 (3/2, 3),
 (3/2, 4),
 (3/2, 9/2),
 (3/2, 6),
 (5/3, 7/5),
 (5/3, 9/5),
 (5/3, 7/3),
 (5/3, 3),
 (5/3, 5),
 (9/5, 5/3),
 (9/5, 3),
 (9/5, 27/5),
 (2, 9/8),
 (2, 8/7),
 (2, 5/4),
 (2, 4/3),
 (2, 3/2),
 (2, 8/5),
 (2, 7/4),
 (2, 9/4),
 (2, 5/2),
 (2, 8/3),
 (2, 3),
 (2, 7/2),
 (2, 9/2),
 (2, 5),
 (2, 6),
 (2, 7),
 (5/2, 5/4),
 (5/2, 8/5),
 (5/2, 2),
 (5/2, 4),
 (5/2, 5),
 (5/2, 10),
 (3, 4/3),
 (3, 3/2),
 (3, 5/3),
 (3, 9/5),
 (3, 2),
 (3, 4),
 (3, 9/2),
 (3, 5),
 (3, 27/5),
 (3, 6),
 (3, 15),
 (4, 8/5),
 (4, 7/4),
 (4, 5/2),
 (4, 7),
 (4, 10),
 (5, 5/3),
 (5, 9/5),
 (5, 3),
 (5, 9),
 (5, 15)}

In [89]:
len(Exception_List2)

75